In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sys; sys.path.insert(0,'..')
from main import *

In [2]:
data = pd.read_csv('../data/spambase/spambase.data')
X = data.iloc[:, :-1].values
y  = data.iloc[:, -1].values

# {0, 1} -> {-1, 1}
y = y * 2 - 1

scaler = MinMaxScaler(feature_range=(0, 1))

for i in range(X.shape[1]):
    X[:, i] = scaler.fit_transform(X[:, i].reshape(-1, 1)).ravel()
    
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, stratify=y, 
                                                    shuffle=True, random_state=1)

# Previous

In [ ]:
# svm_arr, tree_arr = data_size_experiment(X_train, y_train, X_test, y_test,
#                                          experiment_type='table', kernel='linear',
#                                          random_state=1, verbose=True)

In [ ]:
# table_results(svm_arr, tree_arr)

In [ ]:
eq1 = np.sum(y == 1) / y.size
# max(eq1, 1 - eq1)
eq1

# New

In [3]:
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

X_train, y_train, X_test, y_test = X_train[:200], y_train[:200], X_test[:500], y_test[:500]

rank = generate_preliminary_pairs(X_train, y_train, kernel='linear', random_seed=36298)
W = generate_from_preliminary_W(X_train, y_train, rank, m=200, kernel='linear', random_seed=247)

PRELIMINARY
Training accuracy of supervised SVM: 0.82

GENERATE W
[[-1 -1]
 [-1  1]
 [ 1 -1]
 [ 1  1]] [61 49 56 34]
Training accuracy of pairwise rank SVM: 0.96



In [ ]:
# new_svm_lambdaboost(
#     X_train,
#     y_train,
#     X_test,
#     y_test,
#     W,
#     prior=0.99)

In [ ]:
# def new_svm_lambdaboost(X_train, y_train, X_test, y_test, W, prior, T=20, sample_prop=1, random_seed=None, verbose=True):
# """ Generates linear svm lambdaboost classifier """
prior = 0.4
T=20; sample_prop=1; random_seed=None; verbose=True

def class_prior_svm_lambdaboost(X_train, y_train, X_test, y_test, W, prior, T=20, sample_prop=1, random_seed=None, verbose=True):
    """ Generates linear svm lambdaboost classifier using class prior information """

    if verbose:
        print('WITH CLASS PRIOR')
        print('LINEAR SVM SUBMODELS')
        print('Using %d classifiers and sample proportion of %d' % (T, sample_prop))
        if random_seed:
            print('Random seed %d', random_seed)

    # Constants
    m = X_train.shape[0]
    n = X_train.shape[1]
    np.random.seed(random_seed)

    # Initialize model parameters
    f_intercept = 0
    f_coefficient = np.zeros(n)

    # Initialize counters
    t = 1
    alpha = [0.0]
    acc_train_ls = []
    acc_test_ls  = []

    # Training
    while t <= T and alpha[-1] >= 0:

        # Step 6: compute epsilon
        curr_pred = np.dot(X_train, f_coefficient) + f_intercept
        # Scale predictions, works well empirically
        curr_pred = minmax_scale(curr_pred, feature_range=(-1, 1))
        # Remember that W is passed as a list of arrays!
        epsilon = cal_uncertainty(curr_pred, [W])

        # Step 7: compute weights
        weight = cal_weights(epsilon)

        # Step 8: extract labels using class prior
        weight_idx = np.argsort(weight)
        X_new = X_train[weight_idx]
        y_new = np.concatenate((-np.ones(m - int(m * prior)), np.ones(int(m * prior))))

        # Step 9: create training (sample) data by sampling based on weights
        # Sorting weights: necessary for sampling by prob in np.random.choice
        weight = np.sort(weight)
        p_weight = np.abs(weight)
        p_weight /= np.sum(p_weight)
        sample = np.random.choice(m, size=m*sample_prop, replace=True, p=p_weight)
        X_sample = X_new[sample]
        y_sample = y_new[sample]


        # Step 10: learn binary classifier on training (sample) data
        clf = LinearSVC(max_iter=1000)
        clf.fit(X_sample, y_sample)

        # Step 11: predict labels using current classifier
        y_pred = clf.predict(X_train)

        # Step 12: compute weight of current classifier
        alpha_t = cal_alpha(y_pred, epsilon)

        # Make sure alpha is valid
        if np.isnan(alpha_t) or np.isinf(alpha_t):
            print('Alpha invalid, terminated')
            break

        # Step 13: update final classifier
        f_coefficient += alpha_t * clf.coef_.ravel()
        f_intercept += alpha_t * clf.intercept_

        # Update loop
        alpha.append(alpha_t)
        t += 1

        # Evaluation
        y_train_pred = np.dot(X_train, f_coefficient) + f_intercept
        y_test_pred = np.dot(X_test, f_coefficient) + f_intercept
        y_train_pred = np.sign(y_train_pred)
        y_test_pred = np.sign(y_test_pred)

        acc_train_curr = accuracy_score(y_train, y_train_pred)
        acc_test_curr  = accuracy_score(y_test, y_test_pred)
        acc_train_ls.append(max(acc_train_curr, 1 - acc_train_curr))
        acc_test_ls.append(max(acc_test_curr, 1 - acc_test_curr))

        if verbose:
            if t == 2:
                print('t\tPrior\t\tTrain\t\tTest')
            print('%d\t%.2f\t\t%.2f\t\t%.2f' % (t - 1, (np.sum(y_sample == 1) / y_sample.size), acc_train_curr, acc_test_curr))
            if alpha_t < 0:
                print('Alpha %.2f, terminated' % alpha_t)

    # To skip initialized 0 in alpha list
    alpha = alpha[1:]

    # Get final accuracy on best boosting iteration on train set
    # Do not record best iteration on test set -- would train hyperparameter on test
    max_idx = np.argmax(acc_train_ls)
    acc_train_final = acc_train_ls[max_idx]
    acc_test_final  = acc_test_ls[max_idx]

    if verbose:
        print('t = %d was best iteration with accuracy %.2f\n' % (max_idx + 1, acc_test_final))

    # Return minimum error (1 - max accuracy)
    return 1 - acc_train_final, 1 - acc_test_final

In [ ]:
class_prior_svm_lambdaboost(
    X_train,
    y_train,
    X_test,
    y_test,
    W,
    prior,
    T=20,
    sample_prop=1,
    random_seed=None,
    verbose=True,)

In [ ]:
q = np.array([1, 2, 3])
q_new = q[[1, 2]]
q

# Old

In [ ]:
def svm_lambdaboost(X_train, y_train, X_test, y_test, W, T=20, sample_prop=1, random_seed=None, verbose=True):
    """ Generates linear svm lambdaboost classifier """

    if verbose:
        print('LINEAR SVM SUBMODELS')
        print('Using %d classifiers and sample proportion of %d'
              % (T, sample_prop))
        if random_seed:
            print('Random seed %d', random_seed)

    # Constants
    m = X_train.shape[0]
    n = X_train.shape[1]
    np.random.seed(random_seed)

    # Initialize model parameters
    f_intercept = 0
    f_coefficient = np.zeros(n)

    # Initialize counters
    t = 1
    alpha = [0.0]
    acc_train_ls = []
    acc_test_ls  = []

    # Training
    while t <= T and alpha[-1] >= 0:

        # Step 6: compute epsilon
        curr_pred = np.dot(X_train, f_coefficient) + f_intercept
        # Scale predictions, works well empirically
        curr_pred = minmax_scale(curr_pred, feature_range=(-1, 1))
        # Remember that W is passed as a list of arrays!
        epsilon = cal_uncertainty(curr_pred, [W])

        # Step 7: compute weights
        weight = cal_weights(epsilon)

        # Step 8: extract labels
        y = np.sign(weight)

        # Step 9: create training (sample) data by sampling based on weights
        p_weight = np.abs(weight)
        p_weight /= np.sum(p_weight)
        sample = np.random.choice(m, size=m*sample_prop, replace=True, p=p_weight)
        X_sample = X_train[sample]
        y_sample = y[sample]

        # Step 10: learn binary classifier on training (sample) data
        clf = LinearSVC(max_iter=1000)
        clf.fit(X_sample, y_sample)

        # Step 11: predict labels using current classifier
        y_pred = clf.predict(X_train)

        # Step 12: compute weight of current classifier
        alpha_t = cal_alpha(y_pred, epsilon)

        # Make sure alpha is valid
        if np.isnan(alpha_t) or np.isinf(alpha_t):
            print('Alpha invalid, terminated')
            break

        # Step 13: update final classifier
        f_coefficient += alpha_t * clf.coef_.ravel()
        f_intercept += alpha_t * clf.intercept_

        # Update loop
        alpha.append(alpha_t)
        t += 1

        # Evaluation
        y_train_pred = np.dot(X_train, f_coefficient) + f_intercept
        y_test_pred = np.dot(X_test, f_coefficient) + f_intercept
        y_train_pred = np.sign(y_train_pred)
        y_test_pred = np.sign(y_test_pred)

        acc_train_curr = accuracy_score(y_train, y_train_pred)
        acc_test_curr  = accuracy_score(y_test, y_test_pred)
        acc_train_ls.append(max(acc_train_curr, 1 - acc_train_curr))
        acc_test_ls.append(max(acc_test_curr, 1 - acc_test_curr))

        if verbose:
            if t == 2:
                print('t\tTrain\t\tTest\t\tPrior')
            print('%d\t%.2f\t\t%.2f\t\t%.2f' % (t - 1, acc_train_curr, acc_test_curr, (np.sum(y_sample == 1) / y_sample.size)))
            if alpha_t < 0:
                print('Alpha %.2f, terminated' % alpha_t)

    # To skip initialized 0 in alpha list
    alpha = alpha[1:]

    """ CHANGED """
    # # Return final classifier parameters, weight of each submodel
    # return f_coefficient, f_intercept, alpha

    """ CHANGED (again) """
    # acc_train = accuracy_score(y_train, y_train_pred)
    # acc_test  =  accuracy_score(y_test, y_test_pred)
    # return min(acc_train, 1 - acc_train), min(acc_test, 1 - acc_test)

    # Get final accuracy on best boosting iteration on train set
    # Do not record best iteration on test set -- would train hyperparameter on test
    max_idx = np.argmax(acc_train_ls)
    acc_train_final = acc_train_ls[max_idx]
    acc_test_final  = acc_test_ls[max_idx]

    if verbose:
        print('t = %d was best iteration with accuracy %.2f\n' % (max_idx + 1, acc_test_final))

    # Return minimum error (1 - max accuracy)
    return 1 - acc_train_final, 1 - acc_test_final

In [ ]:
svm_lambdaboost(
    X_train,
    y_train,
    X_test,
    y_test,
    W)

In [4]:
tree_lambdaboost(
    X_train,
    y_train,
    X_test,
    y_test,
    W)

DECISION TREE SUBMODELS
Using 20 classifiers and sample proportion of 1
t	Train		Test
1	0.71		0.72
2	0.31		0.30
3	0.70		0.70
4	0.30		0.28
5	0.71		0.71
6	0.71		0.71
7	0.77		0.80
8	0.77		0.76
9	0.76		0.76
10	0.78		0.76
11	0.78		0.77
12	0.79		0.78
13	0.79		0.79
14	0.79		0.80
15	0.80		0.79
16	0.80		0.79
17	0.80		0.80
18	0.80		0.79
19	0.81		0.80
20	0.81		0.80
t = 20 was best iteration with accuracy 0.80



(0.18999999999999995, 0.19599999999999995)

In [11]:
def class_prior_tree_lambdaboost(X_train, y_train, X_test, y_test, W, prior, T=20, max_depth=5, sample_prop=1, 
                                 random_seed=None, verbose=True):
    """ Generates decision tree lambdaboost classifier """

    if verbose:
        print('WITH CLASS PRIOR %.2f' % prior)
        print('DECISION TREE SUBMODELS')
        print('Using %d classifiers and sample proportion of %d' % (T, sample_prop))
        if random_seed:
            print('Random seed %d', random_seed)

    # Constants
    m = X_train.shape[0]
    n = X_train.shape[1]
    np.random.seed(random_seed)

    # Initialize counters
    t = 1
    alpha_t = 0
    acc_train_ls = []
    acc_test_ls  = []

    # Instantiate models and weights
    f = []
    alpha = []

    # Training
    while t <= T and alpha_t >= 0:

        # Step 6: compute epsilon
        if t == 1:
            curr_pred = np.zeros(y_train.shape)
        else:
            curr_pred = sum([alpha[i] * f[i].predict(X_train) for i in range(t - 1)])
        # Scale predictions, works well empirically for SVM...
        #curr_pred = minmax_scale(curr_pred, feature_range=(-1, 1))
        # Remember that W is passed as a list of arrays!
        epsilon = cal_uncertainty(curr_pred, [W])

        # Step 7: compute weights
        weight = cal_weights(epsilon)

        # Step 8: extract labels
        weight_idx = np.argsort(weight)
        X_new = X_train[weight_idx]
        y_new = np.concatenate((-np.ones(m - int(m * prior)), np.ones(int(m * prior))))

        # Step 9: create training (sample) data by sampling based on weights
        # Sorting weights: necessary for sampling by prob in np.random.choice
        weight = np.sort(weight)
        p_weight = np.abs(weight)
        p_weight /= np.sum(p_weight)
        sample = np.random.choice(m, size=m*sample_prop, replace=True, p=p_weight)
        X_sample = X_new[sample]
        y_sample = y_new[sample]

        # Step 10: learn binary classifier on training (sample) data
        clf = DecisionTreeClassifier(max_depth=max_depth)
        clf.fit(X_sample, y_sample)

        # Step 11: predict labels using current classifier
        y_pred = clf.predict(X_train)

        # Step 12: compute weight of current classifier
        alpha_t = cal_alpha(y_pred, epsilon)

        # Make sure alpha is valid
        if np.isnan(alpha_t) or np.isinf(alpha_t):
            print('Alpha invalid, terminated')
            break

        # Step 13: update final classifier
        f.append(clf)

        # Update loop
        alpha.append(alpha_t)
        t += 1

        # Evaluation
        y_train_pred = sum([alpha[i] * f[i].predict(X_train) for i in range(t - 1)])
        y_test_pred = sum([alpha[i] * f[i].predict(X_test) for i in range(t - 1)])
        y_train_pred = np.sign(y_train_pred)
        y_test_pred = np.sign(y_test_pred)

        acc_train_curr = accuracy_score(y_train, y_train_pred)
        acc_test_curr  = accuracy_score(y_test, y_test_pred)
        acc_train_ls.append(max(acc_train_curr, 1 - acc_train_curr))
        acc_test_ls.append(max(acc_test_curr, 1 - acc_test_curr))

        if verbose:
            if t == 2:
                print('t\tPrior\t\tTrain\t\tTest')
            print('%d\t%.2f\t\t%.2f\t\t%.2f' % (t - 1, (np.sum(y_sample == 1) / y_sample.size), 
                                                acc_train_curr, acc_test_curr))
            if alpha_t < 0:
                print('Alpha %.2f, terminated' % alpha_t)

    # Get final accuracy on best boosting iteration on train set
    # Do not record best iteration on test set -- would train hyperparameter on test
    max_idx = np.argmax(acc_train_ls)
    acc_train_final = acc_train_ls[max_idx]
    acc_test_final  = acc_test_ls[max_idx]

    if verbose:
        print('t = %d was best iteration with accuracy %.2f\n' % (max_idx + 1, acc_test_final))

    # Return minimum error (1 - max accuracy)
    return 1 - acc_train_final, 1 - acc_test_final

In [49]:
class_prior_tree_lambdaboost(
    X_train,
    y_train,
    X_test,
    y_test,
    W,
    prior=0.4,
    max_depth=3)

WITH CLASS PRIOR 0.40
DECISION TREE SUBMODELS
Using 20 classifiers and sample proportion of 1
t	Prior		Train		Test
1	0.49		0.78		0.81
2	0.42		0.30		0.23
3	0.48		0.79		0.83
4	0.46		0.79		0.83
5	0.52		0.76		0.78
6	0.55		0.78		0.80
7	0.52		0.77		0.79
8	0.41		0.77		0.80
9	0.56		0.78		0.80
10	0.47		0.77		0.81
11	0.47		0.77		0.82
12	0.46		0.78		0.82
13	0.47		0.78		0.81
14	0.53		0.79		0.81
15	0.42		0.79		0.82
16	0.50		0.79		0.82
17	0.48		0.79		0.82
18	0.52		0.80		0.81
19	0.53		0.80		0.81
20	0.47		0.81		0.81
t = 20 was best iteration with accuracy 0.81



(0.19499999999999995, 0.18799999999999994)